In [ ]:
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio

# Load a grayscale image (512x512)
original_image = cv2.imread('grayscale_image.png', cv2.IMREAD_GRAYSCALE)

# Function to add salt and pepper noise
def add_salt_and_pepper_noise(image, salt_prob, pepper_prob):
    noisy_image = np.copy(image)
    total_pixels = image.size

    # Add salt noise
    salt_pixels = int(total_pixels * salt_prob)
    salt_coords = [np.random.randint(0, i - 1, salt_pixels) for i in image.shape]
    noisy_image[salt_coords[0], salt_coords[1]] = 255

    # Add pepper noise
    pepper_pixels = int(total_pixels * pepper_prob)
    pepper_coords = [np.random.randint(0, i - 1, pepper_pixels) for i in image.shape]
    noisy_image[pepper_coords[0], pepper_coords[1]] = 0

    return noisy_image

# Add salt and pepper noise to the original image
salt_and_pepper_noisy_image = add_salt_and_pepper_noise(original_image, salt_prob=0.01, pepper_prob=0.01)

# Define harmonic mean filter function
def harmonic_mean_filter(image, kernel_size):
    # Apply harmonic mean filter using OpenCV's filter2D
    kernel = np.ones((kernel_size, kernel_size), dtype=np.float32) / (cv2.filter2D(image.astype(np.float32)**(-1), -1, kernel)**(-1))
    filtered_image = cv2.filter2D(image, -1, kernel)
    return filtered_image

# Define geometric mean filter function
def geometric_mean_filter(image, kernel_size):
    # Apply geometric mean filter using OpenCV's filter2D
    kernel = np.ones((kernel_size, kernel_size), dtype=np.float32)
    filtered_image = cv2.pow(image.astype(np.float32), kernel_size*kernel_size)
    filtered_image = cv2.pow(filtered_image, 1/(kernel_size*kernel_size))
    return np.uint8(filtered_image)

# Apply harmonic mean filter to the noisy image
harmonic_filtered_image = harmonic_mean_filter(salt_and_pepper_noisy_image, kernel_size=5)

# Apply geometric mean filter to the noisy image
geometric_filtered_image = geometric_mean_filter(salt_and_pepper_noisy_image, kernel_size=5)

# Calculate PSNR for harmonic mean filter
psnr_harmonic = peak_signal_noise_ratio(original_image, harmonic_filtered_image)

# Calculate PSNR for geometric mean filter
psnr_geometric = peak_signal_noise_ratio(original_image, geometric_filtered_image)

print(f'PSNR for Harmonic Mean Filter: {psnr_harmonic:.2f} dB')
print(f'PSNR for Geometric Mean Filter: {psnr_geometric:.2f} dB')

# Display or save the filtered images and original image
cv2.imshow('Original Image', original_image)
cv2.imshow('Salt & Pepper Noisy Image', salt_and_pepper_noisy_image)
cv2.imshow('Harmonic Mean Filtered Image', harmonic_filtered_image)
cv2.imshow('Geometric Mean Filtered Image', geometric_filtered_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
